In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np

In [2]:
# Booking/Ticket data
df = pd.read_csv('data/registration_data.csv', encoding = 'windows-1252')
df.head()

/tmp/ipykernel_12274/990014732.py:2: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/registration_data.csv', encoding = 'windows-1252')


,FCID,BookingID,BCreatedDateAEST,BStatusAfterSubmitted,BStatusFromDateTimeAEST,BStatusFinal,BTicketID,BTicketType,TaskName,IsCustomService,IsCustomRepair,IsDeleted,PriceIncGSTRaw,VYMM,VMakeModel,VMake,VYear,VMid,VRego,BShopID,BShopPostcode,BShopState,BShopRegionName,BShopRegionClass
0,2,1449822,2024-02-19,33. Approved,2024-02-19 16:01:15,64. Completed,2323471,Log,"Logbook Service - 75,000 km / 60 months",0,0,False,464.27,2019 AUDI A5 COUPE,AUDI A5 COUPE,AUDI,2019.0,AUD45780,YPG67B,17177,2606,ACT,Canberra,2.0
1,2,1565721,2024-03-19,33. Approved,2024-03-19 13:56:15,64. Completed,2537851,Capped,"Capped Price Service - 10,000 km",0,0,False,290.00,2022 TOYOTA HILUX,TOYOTA HILUX,TOYOTA,2022.0,TOY47062,1HSB467,1133,6017,WA,Perth,2.0
2,1,1320451,2023-11-13,33. Approved,2023-11-14 13:55:17,64. Completed,2053300,ProductTyreOp,((Products)),0,0,False,60.50,2001 NISSAN PATROL,NISSAN PATROL,NISSAN,2001.0,NIS29550,DE18YN,14790,2263,NSW,Central Coast,3.0
3,1,1320451,2023-11-13,33. Approved,2023-11-14 13:55:17,64. Completed,2053301,ProductTyre,((Products)),0,0,False,1484.00,2001 NISSAN PATROL,NISSAN PATROL,NISSAN,2001.0,NIS29550,DE18YN,14790,2263,NSW,Central Coast,3.0
4,2,1448252,2024-02-19,33. Approved,2024-02-19 11:30:01,64. Completed,2320054,Capped,"Capped Price Service - 105,000 km",0,0,False,358.00,2019 HYUNDAI SANTA FE,HYUNDAI SANTA FE,HYUNDAI,2019.0,HYU45824,1RE6XD,17962,3844,VIC,Traralgon,2.0


In [3]:
columns_to_keep = ['VRego', 'VMakeModel', 'VMake', 'VMid', 'VYear']  # replace with actual column names
df = df[columns_to_keep]
df.head()

,VRego,VMakeModel,VMake,VMid,VYear
0,YPG67B,AUDI A5 COUPE,AUDI,AUD45780,2019.0
1,1HSB467,TOYOTA HILUX,TOYOTA,TOY47062,2022.0
2,DE18YN,NISSAN PATROL,NISSAN,NIS29550,2001.0
3,DE18YN,NISSAN PATROL,NISSAN,NIS29550,2001.0
4,1RE6XD,HYUNDAI SANTA FE,HYUNDAI,HYU45824,2019.0


In [4]:
df2 = pd.read_excel('data/mid_data.xlsx', engine='openpyxl')
df2.head()

,VehicleMid,countAGTixForMid,Manufacturer Name,Model Name,Fuel Type,EngineCC,Transmission,Wheel Drive,countFMORegos,minVehicleYear,maxVehicleYear
0,FOR43446,53116,FORD,RANGER,Diesel,3.2,Automatic,Four Wheel Drive,436,2015,2021
1,FOR43446,53116,FORD,RANGER,Diesel,2.0,Automatic,Four Wheel Drive,61,2020,2024
2,FOR43446,53116,FORD,RANGER,Diesel,2.2,Automatic,Four Wheel Drive,21,2018,2021
3,FOR43446,53116,FORD,RANGER,Diesel,3.2,Automatic,Rear Wheel Drive,16,2018,2021
4,FOR43446,53116,FORD,RANGER,Diesel,3.2,Manual,Four Wheel Drive,9,2018,2021


In [5]:
columns_to_keep = ['VehicleMid', 'Fuel Type', 'EngineCC', 'Transmission', 'Wheel Drive'] 
df2 = df2[columns_to_keep]
df2 = df2.rename(columns={
    'VehicleMid': 'VMid',
    'Fuel Type': 'FuelType',
    'EngineCC': 'EngineSize',
    'Wheel Drive': 'DriveType',
    })
df2.head()

,VMid,FuelType,EngineSize,Transmission,DriveType
0,FOR43446,Diesel,3.2,Automatic,Four Wheel Drive
1,FOR43446,Diesel,2.0,Automatic,Four Wheel Drive
2,FOR43446,Diesel,2.2,Automatic,Four Wheel Drive
3,FOR43446,Diesel,3.2,Automatic,Rear Wheel Drive
4,FOR43446,Diesel,3.2,Manual,Four Wheel Drive


In [6]:
merged_df = pd.merge(df, df2, on='VMid', how='inner')
merged_df = merged_df.drop_duplicates(subset='VRego', keep='last')

In [7]:
merged_df = merged_df.rename(columns={
    'VRego': 'Registration',
    'VMakeModel': 'Model',
    'VMake': 'Make',
    'VYear': 'Year',
    })

In [8]:
merged_df.head()

,Registration,Model,Make,VMid,Year,FuelType,EngineSize,Transmission,DriveType
3,1HSB467,TOYOTA HILUX,TOYOTA,TOY47062,2022.0,Diesel,2.4,Automatic,Rear Wheel Drive
41,782CZ9,TOYOTA RAV4,TOYOTA,TOY45521,2021.0,Petrol,2.5,CVT,Front Wheel Drive
61,CE14UX,SUZUKI BALENO,SUZUKI,SUZ40295,2020.0,Petrol,1.4,Automatic,Front Wheel Drive
69,273YSV,TOYOTA HILUX,TOYOTA,TOY39674,2018.0,Diesel,2.8,Automatic,Four Wheel Drive
198,CN72QV,ISUZU D-MAX,ISUZU,ISU30317,2017.0,Diesel,3.0,Automatic,Rear Wheel Drive


In [9]:
mapping = {
    "DSG": "Auto",
    "CVT": "Auto",
    "Automatic": "Auto",
    "Manual": "Manual"
}
merged_df["Transmission"] = merged_df["Transmission"].map(mapping)

merged_df['Transmission'].unique()

array(['Auto', 'Manual'], dtype=object)

In [10]:
mapping = {
    "Front Wheel Drive": "2WD",
    "Rear Wheel Drive": "2WD",
    "Four Wheel Drive": "4WD",

}
merged_df["DriveType"] = merged_df["DriveType"].map(mapping)

merged_df['DriveType'].unique()

array(['2WD', '4WD'], dtype=object)

In [11]:
import os

# Define paths
paths = {
    "rego": "../backend/data/rego_data.csv",
}

# Make sure the directories exist
for path in paths.values():
    dir_path = os.path.dirname(path)
    os.makedirs(dir_path, exist_ok=True)  

# Save CSVs
merged_df.to_csv(paths["rego"], index=False)

print("All files saved successfully:")
for name, path in paths.items():
    print(f"{name}: {os.path.abspath(path)}")

All files saved successfully:
rego: /home/dylan/git/CAPSTONE_T316/backend/data/rego_data.csv
